# Setup and Imports

In [1]:
import os
import sys
import json
import asyncio
import subprocess
from pathlib import Path
from typing import List, Dict, Any, Optional
from datetime import datetime
from dataclasses import dataclass, asdict
import numpy as np

# Add backend to path
sys.path.insert(0, str(Path.cwd()))

# Load environment
from dotenv import load_dotenv
load_dotenv('.env.local')

# Metrics imports
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize

try:
    from rouge_score import rouge_scorer
except ImportError:
    print("⚠ rouge_score not installed. Run: pip install rouge-score")

try:
    from nltk.translate.meteor_score import meteor_score
except ImportError:
    print("⚠ METEOR requires NLTK meteor scorer")

try:
    from bert_score import score as bert_score_fn
except ImportError:
    print("⚠ bert_score not installed. Run: pip install bert-score")

try:
    from openai import OpenAI
except ImportError:
    print("⚠ openai not installed. Run: pip install openai")

# Download NLTK data if needed
try:
    import nltk
    nltk.data.find('tokenizers/punkt')
except LookupError:
    print("Downloading NLTK data...")
    nltk.download('punkt', quiet=True)
    nltk.download('wordnet', quiet=True)

# ============================================================================
# 🔌 DATABASE CONNECTION (via docker-compose exec) - SAMA SEPERTI rag_document_pipeline.ipynb
# ============================================================================

class DatabaseConnection:
    """Docker PostgreSQL connection via docker-compose exec with stdin"""
    def __init__(self, working_dir: str = "."):
        self.working_dir = working_dir
        self.container_name = "system-llm-postgres-local"
        self.user = "llm_user"
        self.db = "system_llm"

    def execute_sql(self, query: str, fetch: bool = False) -> str:
        """Execute SQL via docker-compose exec with stdin (handles long queries)"""
        cmd = [
            "docker-compose", "-f", "docker-compose.local.yml",
            "exec", "-T", "postgres",
            "psql", "-U", self.user, "-d", self.db,
        ]
        
        if fetch:
            cmd.append("-t")

        try:
            result = subprocess.run(
                cmd,
                input=query,
                capture_output=True,
                text=True,
                encoding='utf-8',
                errors='replace',
                cwd=self.working_dir
            )

            if result.returncode != 0:
                error_msg = result.stderr.strip() if result.stderr else "Unknown error"
                raise Exception(f"SQL Error: {error_msg}")

            return result.stdout.strip() if fetch else ""

        except Exception as e:
            print(f"Database Error: {e}")
            raise

    def test_connection(self) -> bool:
        """Test database connection"""
        try:
            version = self.execute_sql("SELECT version();", fetch=True)
            if version:
                db_version = version.split(',')[0].strip()
                print(f"Connected to {self.container_name}")
                print(f"Version: {db_version}\n")
                return True
        except Exception as e:
            print(f"Connection failed: {e}\n")
            return False

# Initialize database
db = DatabaseConnection(working_dir=".")
print("=" * 80)
print("DATABASE CONNECTION TEST")
print("=" * 80)

if not db.test_connection():
    sys.exit(1)

# OpenAI client
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    print("ERROR: OPENAI_API_KEY not set in .env.local")
    sys.exit(1)

client = OpenAI(api_key=OPENAI_API_KEY)

print("✓ Environment loaded")
print(f"✓ OpenAI API: Ready")

DATABASE CONNECTION TEST
Connected to system-llm-postgres-local
Version: PostgreSQL 15.15 (Debian 15.15-1.pgdg12+1) on x86_64-pc-linux-gnu

✓ Environment loaded
✓ OpenAI API: Ready


# List Models

In [2]:
def list_models() -> List[Dict[str, Any]]:
    """
    Query database for available models.
    """
    try:
        query = """
        SELECT id, name, provider, "order" 
        FROM model 
        ORDER BY "order" ASC;
        """
        result = db.execute_sql(query, fetch=True)
        
        model_list = []
        print(f"\n📊 Available Models:")
        print("=" * 70)
        
        if result:
            for line in result.split('\n'):
                if '|' in line and not line.startswith('-'):
                    parts = line.split('|')
                    if len(parts) >= 4:
                        try:
                            model_id = parts[0].strip()
                            name = parts[1].strip()
                            provider = parts[2].strip()
                            order = parts[3].strip()
                            
                            if model_id and name:
                                model_info = {
                                    "id": model_id,
                                    "name": name,
                                    "provider": provider,
                                    "order": order,
                                }
                                model_list.append(model_info)
                                print(f"{order}. {name:30} | Provider: {provider:15}")
                        except:
                            pass
        
        print("=" * 70)
        if model_list:
            print(f"Total: {len(model_list)} models\n")
        return model_list
    except Exception as e:
        print(f"Error listing models: {e}")
        return []

# List available models
available_models = list_models()
if available_models:
    print(f"✓ Models loaded. Using default: {available_models[0]['name']}")
else:
    print("✗ No models found in database")


📊 Available Models:
0. gpt-4.1-nano                   | Provider: OPENAI         
1. claude-haiku-4-5               | Provider: ANTHROPIC      
2. gemini-2.5-flash               | Provider: GOOGLE         
10. meta-llama/llama-3.1-8b-instruct | Provider: openrouter     
11. qwen/qwen-2.5-7b-instruct      | Provider: openrouter     
12. microsoft/phi-3-medium-128k-instruct | Provider: openrouter     
Total: 6 models

✓ Models loaded. Using default: gpt-4.1-nano


# EvaluationChatService

In [3]:
@dataclass
class EvaluationConfig:
    """Configuration for evaluation run - matches ChatSession fields."""
    model_name: str = "gpt-4.1-nano"
    use_tools: bool = True
    with_rag: bool = True
    max_tool_iterations: int = 3
    temperature: float = 0.7
    
    # Override fields (like ChatSession can override per-session)
    prompt_general_override: Optional[str] = None
    task_override: Optional[str] = None
    persona_override: Optional[str] = None
    mission_objective_override: Optional[str] = None
    prompt_id_override: Optional[str] = None

# ============================================================================
# Helper Functions - Load from Database (matches production)
# ============================================================================

def get_chat_config() -> Dict[str, Any]:
    """Get ChatConfig singleton from database (id=1)."""
    try:
        query = """
        SELECT id, prompt_general, prompt_refine, prompt_analysis, 
               default_top_k, max_top_k, similarity_threshold,
               tool_calling_max_iterations, tool_calling_enabled,
               include_rag_instruction
        FROM chat_config WHERE id = 1;
        """
        result = db.execute_sql(query, fetch=True)
        
        if result:
            # Parse psql output
            for line in result.split('\n'):
                if '|' in line and not line.startswith('-'):
                    parts = line.split('|')
                    if len(parts) >= 10:
                        return {
                            "id": parts[0].strip(),
                            "prompt_general": parts[1].strip() if parts[1].strip() != "" else None,
                            "prompt_refine": parts[2].strip() if parts[2].strip() != "" else None,
                            "prompt_analysis": parts[3].strip() if parts[3].strip() != "" else None,
                            "default_top_k": int(parts[4].strip()) if parts[4].strip() else 5,
                            "max_top_k": int(parts[5].strip()) if parts[5].strip() else 10,
                            "similarity_threshold": float(parts[6].strip()) if parts[6].strip() else 0.7,
                            "tool_calling_max_iterations": int(parts[7].strip()) if parts[7].strip() else 10,
                            "tool_calling_enabled": bool(int(parts[8].strip())) if parts[8].strip() else True,
                            "include_rag_instruction": bool(int(parts[9].strip())) if parts[9].strip() else True,
                        }
        
        # Return defaults if not found
        return {
            "id": "1",
            "prompt_general": None,
            "prompt_refine": "Refine the student's question to be more specific and clear for better search results.",
            "prompt_analysis": None,
            "default_top_k": 5,
            "max_top_k": 10,
            "similarity_threshold": 0.7,
            "tool_calling_max_iterations": 10,
            "tool_calling_enabled": True,
            "include_rag_instruction": True,
        }
    except Exception as e:
        print(f"Warning: Could not get ChatConfig from DB: {e}")
        return {
            "prompt_general": None,
            "prompt_refine": "Refine the student's question to be more specific and clear.",
            "prompt_analysis": None,
            "default_top_k": 5,
            "max_top_k": 10,
            "similarity_threshold": 0.7,
            "tool_calling_max_iterations": 10,
            "tool_calling_enabled": True,
            "include_rag_instruction": True,
        }

def get_prompt_content(prompt_id: str) -> Optional[str]:
    """Get prompt content from Prompt table."""
    try:
        query = f"SELECT content FROM prompt WHERE id = '{prompt_id}' LIMIT 1;"
        result = db.execute_sql(query, fetch=True)
        
        if result:
            for line in result.split('\n'):
                if line and not line.startswith('-'):
                    return line.strip()
        return None
    except Exception as e:
        print(f"Warning: Could not get prompt from DB: {e}")
        return None

def generate_embedding(text: str) -> List[float]:
    """Generate 1536-dimensional embedding"""
    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )
    return response.data[0].embedding

def semantic_search(query_text: str, top_k: int = 5) -> list:
    """Semantic search using cosine similarity dengan database"""
    try:
        # Generate query embedding
        query_embedding = np.array(generate_embedding(query_text))
        
        # Get chunks from database via SQL (matches production query)
        search_query = """
        SELECT dc.content, d.original_filename, dc.page_number, dc.embedding
        FROM document_chunk dc
        JOIN document d ON dc.document_id = d.id
        WHERE d.status = 'PROCESSED'
        LIMIT 100;
        """
        result_text = db.execute_sql(search_query, fetch=True)
        
        similarities = []
        if result_text:
            for line in result_text.split('\n'):
                if '|' in line:
                    parts = line.split('|', 3)
                    if len(parts) >= 4:
                        try:
                            content = parts[0].strip()
                            filename = parts[1].strip()
                            page_num = int(parts[2].strip()) if parts[2].strip().isdigit() else 0
                            embedding_json = parts[3].strip()
                            
                            if embedding_json.startswith('['):
                                chunk_embedding = np.array(json.loads(embedding_json))
                                similarity = np.dot(query_embedding, chunk_embedding) / (
                                    np.linalg.norm(query_embedding) * np.linalg.norm(chunk_embedding) + 1e-10
                                )
                                similarities.append({
                                    "content": content[:500],
                                    "original_filename": filename,
                                    "page_number": page_num,
                                    "similarity_score": float(similarity)
                                })
                        except:
                            pass
        
        similarities.sort(key=lambda x: x['similarity_score'], reverse=True)
        return similarities[:top_k]
    
    except Exception as e:
        print(f"Semantic search error: {e}")
        return []

class EvaluationChatService:
    """
    Chat service for evaluation - mengikuti arsitektur ChatService asli
    dengan proper refine_prompt dan semantic_search implementation
    """
    
    def __init__(self):
        self.chat_config = get_chat_config()
    
    def _get_tools(self) -> List[Dict[str, Any]]:
        """Get available tools for the chat (matches production order)."""
        return [
            {
                "name": "refine_prompt",
                "description": "Refine and clarify an ambiguous or unclear student question. Use this tool when the student's question is vague, incomplete, or could be interpreted multiple ways.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "original_prompt": {"type": "string", "description": "The original student question that needs refinement/clarification"},
                    },
                    "required": ["original_prompt"]
                }
            },
            {
                "name": "semantic_search",
                "description": "Search for relevant documents based on semantic similarity",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {"type": "string", "description": "Search query"},
                        "top_k": {"type": "integer", "description": "Number of results", "default": 5},
                    },
                    "required": ["query"]
                }
            }
        ]
    
    def build_system_prompt(
        self, 
        config: EvaluationConfig,
        user_task: str = "",
        user_persona: str = "",
        user_mission_objective: str = ""
    ) -> tuple[str, Dict[str, str]]:
        """
        Build system prompt following exact ChatService._build_conversation_context order.
        Returns: (final_system_prompt, prompt_sources_dict)
        """
        prompt_sections = []
        prompt_sources = {
            "prompt_general": None,
            "student_profile": {},
            "specific_prompt": None,
        }
        
        # SECTION 1: General Prompt (highest priority)
        prompt_general = config.prompt_general_override
        if prompt_general is None:
            prompt_general = self.chat_config.get("prompt_general")
        
        if prompt_general:
            prompt_sections.append(f"# General Prompt\n{prompt_general}")
            prompt_sources["prompt_general"] = prompt_general
        
        # SECTION 2: Student Learning Profile (mid priority)
        student_profile = []
        
        task = config.task_override if config.task_override else user_task
        if task:
            student_profile.append(f"# Task\n{task}")
            prompt_sources["student_profile"]["task"] = task
        
        persona = config.persona_override if config.persona_override else user_persona
        if persona:
            student_profile.append(f"# Persona\n{persona}")
            prompt_sources["student_profile"]["persona"] = persona
        
        mission_objective = config.mission_objective_override if config.mission_objective_override else user_mission_objective
        if mission_objective:
            student_profile.append(f"# Mission Objective\n{mission_objective}")
            prompt_sources["student_profile"]["mission_objective"] = mission_objective
        
        if student_profile:
            prompt_sections.append("## Student Learning Profile\n" + "\n\n".join(student_profile))
        
        # SECTION 3: Specific Prompt (lowest priority, from Prompt table)
        if config.prompt_id_override:
            specific_prompt = get_prompt_content(config.prompt_id_override)
            if specific_prompt:
                prompt_sections.append(f"# Specific Prompt\n{specific_prompt}")
                prompt_sources["specific_prompt"] = specific_prompt
        
        system_prompt = "\n\n".join(prompt_sections) if prompt_sections else "You are a helpful educational assistant."
        
        return system_prompt, prompt_sources
    
    async def _execute_refine_prompt(self, args: Dict[str, Any], print_steps: bool = False) -> Dict[str, Any]:
        """
        Execute refine_prompt tool - uses ChatConfig.prompt_refine as instruction.
        THIS IS THE MOST CRITICAL PART - MUST ACTUALLY CALL LLM
        """
        original_prompt = args.get("original_prompt", "")
        
        if not original_prompt:
            if print_steps:
                print(f"\n      ❌ ERROR: Empty prompt provided")
            return {
                "original": original_prompt,
                "refined": original_prompt,
                "success": False,
                "error": "Empty prompt provided"
            }
        
        try:
            # CRITICAL: Get refine instruction from ChatConfig
            refine_instruction = self.chat_config.get("prompt_refine")
            
            if not refine_instruction:
                refine_instruction = "Refine the student's question to be more specific and clear for better search results."
            
            if print_steps:
                print(f"\n      ✅ [STEP 1] Got refine instruction from ChatConfig")
                print(f"      {'-'*70}")
                print(f"      Instruction: {refine_instruction}")
                print(f"      {'-'*70}")
                print(f"\n      ✅ [STEP 2] Building messages for LLM")
            
            # Create messages for LLM refinement (THIS IS CRITICAL)
            messages = [
                {
                    "role": "system",
                    "content": refine_instruction
                },
                {
                    "role": "user",
                    "content": original_prompt
                }
            ]
            
            if print_steps:
                print(f"      System message: {messages[0]['content'][:80]}...")
                print(f"      User message: {messages[1]['content']}")
                print(f"\n      ✅ [STEP 3] Calling OpenAI API with gpt-4.1-nano")
            
            # CRITICAL: Call OpenAI API - this MUST execute and return actual LLM output
            try:
                response = client.chat.completions.create(
                    model="gpt-4.1-nano",  # Use this fast model like production
                    messages=messages,
                    temperature=0.7,
                    max_tokens=300
                )
                
                refined_prompt = response.choices[0].message.content
                
                if not refined_prompt:
                    refined_prompt = original_prompt
                
                if print_steps:
                    print(f"      ✅ [STEP 4] Got response from OpenAI")
                    print(f"      {'-'*70}")
                    print(f"      Original: {original_prompt}")
                    print(f"      Refined:  {refined_prompt}")
                    print(f"      {'-'*70}\n")
                
                return {
                    "original": original_prompt,
                    "refined": refined_prompt,
                    "success": True
                }
            
            except Exception as api_error:
                if print_steps:
                    print(f"      ❌ [STEP 3] API ERROR: {str(api_error)}")
                    print(f"      Returning original unchanged (fallback)\n")
                
                # On API error, return original
                return {
                    "original": original_prompt,
                    "refined": original_prompt,
                    "success": False,
                    "error": f"API Error: {str(api_error)}"
                }
        
        except Exception as e:
            if print_steps:
                print(f"      ❌ [STEP ?] UNEXPECTED ERROR: {str(e)}")
                print(f"      Type: {type(e).__name__}\n")
            
            return {
                "original": original_prompt,
                "refined": original_prompt,
                "success": False,
                "error": f"Unexpected error: {str(e)}"
            }
    
    async def _execute_semantic_search(self, args: Dict[str, Any], print_steps: bool = False) -> Dict[str, Any]:
        """Execute semantic search with detailed logging."""
        query = args.get("query", "")
        top_k = args.get("top_k", self.chat_config.get("default_top_k", 5))
        
        try:
            results = semantic_search(query, top_k=top_k)
            
            if print_steps:
                print(f"\n      [semantic_search RESULTS]")
                print(f"      Query: {query}")
                print(f"      Top-K: {top_k}")
                print(f"      Found: {len(results)} documents\n")
                for i, r in enumerate(results, 1):
                    print(f"      [{i}] {r['original_filename']} (page {r['page_number']}, similarity: {r['similarity_score']:.3f})")
                    print(f"          {r['content'][:150]}...\n")
            
            return {
                "query": query,
                "results": [
                    {
                        "content": r.get("content", ""),
                        "filename": r.get("original_filename"),
                        "page": r.get("page_number"),
                        "similarity": r.get("similarity_score"),
                    }
                    for r in results
                ]
            }
        except Exception as e:
            return {"error": str(e), "query": query}
    
    async def send_message_with_tools(
        self,
        user_message: str,
        config: EvaluationConfig,
        user_task: str = "",
        user_persona: str = "",
        user_mission_objective: str = "",
        print_steps: bool = True
    ) -> Dict[str, Any]:
        """Send message with tool calling - matches production flow."""
        
        # Build system prompt with all layers
        system_prompt, prompt_sources = self.build_system_prompt(
            config, user_task, user_persona, user_mission_objective
        )
        
        if print_steps:
            print(f"\n{'='*70}")
            print(f"📋 SYSTEM PROMPT CONSTRUCTION")
            print(f"{'='*70}")
            print(system_prompt)
            print(f"{'='*70}\n")
        
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
        
        tools = self._get_tools()
        
        full_response = ""
        tool_calls_made = []
        iteration = 0
        max_iterations = config.max_tool_iterations
        
        while iteration < max_iterations:
            iteration += 1
            print(f"  [Iteration {iteration}/{max_iterations}]", end=" ")
            
            # Call OpenAI dengan tools
            response = client.chat.completions.create(
                model=config.model_name,
                messages=messages,
                tools=[{"type": "function", "function": t} for t in tools],
                tool_choice="auto",
                temperature=config.temperature
            )
            
            assistant_message = response.choices[0].message
            
            if assistant_message.tool_calls:
                # Add assistant message with tool_calls to messages
                tool_calls = assistant_message.tool_calls
                messages.append({
                    "role": "assistant",
                    "content": assistant_message.content or "",
                    "tool_calls": [
                        {
                            "id": tc.id,
                            "type": "function",
                            "function": {
                                "name": tc.function.name,
                                "arguments": tc.function.arguments
                            }
                        }
                        for tc in tool_calls
                    ]
                })
                
                # Execute tools
                for tool_call in tool_calls:
                    print(f"🔧 {tool_call.function.name}", end=" ")
                    tool_name = tool_call.function.name
                    try:
                        tool_args = json.loads(tool_call.function.arguments)
                        
                        # Execute the appropriate tool with detailed logging
                        if tool_name == "refine_prompt":
                            result = await self._execute_refine_prompt(tool_args, print_steps=print_steps)
                        elif tool_name == "semantic_search":
                            result = await self._execute_semantic_search(tool_args, print_steps=print_steps)
                        else:
                            result = {"error": f"Unknown tool: {tool_name}"}
                        
                        # Add tool result to messages
                        messages.append({
                            "role": "tool",
                            "tool_call_id": tool_call.id,
                            "content": json.dumps(result)
                        })
                        
                        tool_calls_made.append({
                            "name": tool_name,
                            "args": tool_args,
                            "result": result,
                        })
                        
                        print(f"→ OK", end=" ")
                    except Exception as e:
                        print(f"✗ Error: {e}", end=" ")
                
                print()
            else:
                # No tool calls, we're done
                full_response = assistant_message.content or ""
                print("✓ Done")
                break
        
        return {
            "content": full_response,
            "tool_calls": tool_calls_made,
            "iteration": iteration,
            "prompt_sources": prompt_sources,
        }
    
    async def send_message_stream(
        self,
        user_message: str,
        config: EvaluationConfig,
        user_task: str = "",
        user_persona: str = "",
        user_mission_objective: str = "",
        print_steps: bool = True
    ) -> Dict[str, Any]:
        """Send message without tools."""
        
        # Build system prompt with all layers
        system_prompt, prompt_sources = self.build_system_prompt(
            config, user_task, user_persona, user_mission_objective
        )
        
        if print_steps:
            print(f"\n{'='*70}")
            print(f"📋 SYSTEM PROMPT CONSTRUCTION")
            print(f"{'='*70}")
            print(system_prompt)
            print(f"{'='*70}\n")
        
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
        
        response = client.chat.completions.create(
            model=config.model_name,
            messages=messages,
            temperature=config.temperature
        )
        
        return {
            "content": response.choices[0].message.content or "",
            "tool_calls": [],
            "iteration": 1,
            "prompt_sources": prompt_sources,
        }

print("✓ EvaluationChatService ready (with fixed refine_prompt execution)")

✓ EvaluationChatService ready (with fixed refine_prompt execution)


# EvaluationMetrics

In [4]:
class EvaluationMetrics:
    """
    Evaluation metrics for chat responses:
    - BLEU: N-gram precision
    - ROUGE-L: Longest common subsequence F1
    - METEOR: Alignment-based with stemming
    - BERTScore: Semantic similarity (Precision, Recall, F1)
    """
    
    @staticmethod
    def bleu_score(reference: str, candidate: str, weights: tuple = (0.25, 0.25, 0.25, 0.25)) -> float:
        """
        Calculate BLEU score (n-gram precision).
        Range: [0, 1] where 1 is perfect match.
        """
        try:
            reference_tokens = word_tokenize(reference.lower())
            candidate_tokens = word_tokenize(candidate.lower())
            smoothing_function = SmoothingFunction().method1
            score = sentence_bleu(
                [reference_tokens],
                candidate_tokens,
                weights=weights,
                smoothing_function=smoothing_function
            )
            return float(score)
        except Exception as e:
            print(f"Warning: BLEU score calculation failed: {e}")
            return 0.0
    
    @staticmethod
    def rouge_l_score(reference: str, candidate: str) -> float:
        """
        Calculate ROUGE-L score (longest common subsequence F1).
        Range: [0, 1] where 1 is perfect match.
        """
        try:
            scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
            scores = scorer.score(reference, candidate)
            return float(scores['rougeL'].fmeasure)
        except Exception as e:
            print(f"Warning: ROUGE-L calculation failed: {e}")
            return 0.0
    
    @staticmethod
    def meteor_score(reference: str, candidate: str) -> float:
        """
        Calculate METEOR score (alignment-based with stemming).
        Range: [0, 1] where 1 is perfect match.
        """
        try:
            reference_tokens = word_tokenize(reference.lower())
            candidate_tokens = word_tokenize(candidate.lower())
            score = meteor_score([reference_tokens], candidate_tokens)
            return float(score)
        except Exception as e:
            print(f"Warning: METEOR calculation failed: {e}")
            return 0.0
    
    @staticmethod
    def bert_score(reference: str, candidate: str, model_type: str = "distilbert-base-uncased") -> Dict[str, float]:
        """
        Calculate BERTScore (semantic similarity).
        Returns: {"precision": float, "recall": float, "f1": float}
        Range: [0, 1] where 1 is perfect match.
        """
        try:
            P, R, F1 = bert_score_fn([candidate], [reference], model_type=model_type, verbose=False)
            return {
                "precision": float(P[0]),
                "recall": float(R[0]),
                "f1": float(F1[0])
            }
        except Exception as e:
            print(f"Warning: BERTScore calculation failed: {e}")
            return {"precision": 0.0, "recall": 0.0, "f1": 0.0}
    
    @staticmethod
    def calculate_all_metrics(reference: str, candidate: str) -> Dict[str, Any]:
        """
        Calculate all evaluation metrics.
        """
        print("  📊 Computing metrics:", end=" ")
        
        metrics = {}
        
        print("BLEU", end=" ")
        metrics["bleu"] = EvaluationMetrics.bleu_score(reference, candidate)
        
        print("ROUGE-L", end=" ")
        metrics["rouge_l"] = EvaluationMetrics.rouge_l_score(reference, candidate)
        
        print("METEOR", end=" ")
        metrics["meteor"] = EvaluationMetrics.meteor_score(reference, candidate)
        
        print("BERTScore", end=" ")
        metrics["bert_score"] = EvaluationMetrics.bert_score(reference, candidate)
        
        # Calculate length ratio
        ref_len = len(reference.split())
        cand_len = len(candidate.split())
        metrics["length_ratio"] = cand_len / ref_len if ref_len > 0 else 0.0
        
        print("✓")
        
        return metrics

print("✓ EvaluationMetrics ready")

✓ EvaluationMetrics ready


# EvaluationRunner

In [5]:
class EvaluationRunner:
    """Orchestrates evaluation runs - matches production flow."""
    
    def __init__(self):
        self.chat_service = EvaluationChatService()
    
    async def evaluate_response(
        self,
        config: EvaluationConfig,
        test_question: str,
        reference_answer: str,
        user_task: str = "",
        user_persona: str = "",
        user_mission_objective: str = "",
        print_steps: bool = True,
    ) -> Dict[str, Any]:
        """Evaluate a single response with full context."""
        
        print(f"\n{'='*70}")
        print(f"🧪 Evaluation Run")
        print(f"{'='*70}")
        print(f"Model: {config.model_name}")
        print(f"With Tools: {config.use_tools}")
        print(f"Question: {test_question[:80]}...")
        
        if user_task:
            print(f"Task: {user_task[:60]}...")
        if user_persona:
            print(f"Persona: {user_persona[:60]}...")
        if user_mission_objective:
            print(f"Mission: {user_mission_objective[:60]}...")
        
        print(f"{'-'*70}")
        
        try:
            print(f"📤 Sending to {config.model_name}...")
            if config.use_tools:
                if print_steps:
                    print("\n📋 Evaluation Mode: WITH TOOLS (agentic loop)\n")
                response = await self.chat_service.send_message_with_tools(
                    user_message=test_question,
                    config=config,
                    user_task=user_task,
                    user_persona=user_persona,
                    user_mission_objective=user_mission_objective,
                    print_steps=print_steps
                )
            else:
                if print_steps:
                    print("\n📋 Evaluation Mode: WITHOUT TOOLS (direct response)\n")
                response = await self.chat_service.send_message_stream(
                    user_message=test_question,
                    config=config,
                    user_task=user_task,
                    user_persona=user_persona,
                    user_mission_objective=user_mission_objective,
                    print_steps=print_steps
                )
            
            response_text = response.get("content", "")
            tool_calls = response.get("tool_calls", [])
            
            print(f"\n✓ Response received ({len(response_text)} chars)")
            print(f"  Tool calls made: {len(tool_calls)}")
            
            if print_steps and tool_calls:
                print("\n🔧 Tool Calls Summary:")
                for i, tc in enumerate(tool_calls, 1):
                    print(f"\n  [{i}] Tool: {tc['name']}")
                    if tc['name'] == 'refine_prompt':
                        print(f"      Original: {tc['args'].get('original_prompt', '')[:100]}")
                        print(f"      Refined:  {tc['result'].get('refined', '')[:100]}")
                    elif tc['name'] == 'semantic_search':
                        print(f"      Query: {tc['args'].get('query', '')}")
                        print(f"      Results: {len(tc['result'].get('results', []))} documents found")
            
            if print_steps:
                print(f"\n📝 Final Response from LLM:")
                print(f"{'-'*70}")
                print(response_text[:500] + ("..." if len(response_text) > 500 else ""))
                print(f"{'-'*70}\n")
            
            metrics = EvaluationMetrics.calculate_all_metrics(reference_answer, response_text)
            
            result = {
                "timestamp": datetime.now().isoformat(),
                "config": asdict(config),
                "question": test_question,
                "response": response_text,
                "reference_answer": reference_answer,
                "user_context": {
                    "task": user_task,
                    "persona": user_persona,
                    "mission_objective": user_mission_objective,
                },
                "metrics": metrics,
                "tool_calls": tool_calls,
            }
            
            print(f"\n📊 Results:")
            print(f"  BLEU:       {metrics['bleu']:.4f}")
            print(f"  ROUGE-L:    {metrics['rouge_l']:.4f}")
            print(f"  METEOR:     {metrics['meteor']:.4f}")
            print(f"  BERTScore:  {metrics['bert_score']['f1']:.4f}")
            
            return result
        
        except Exception as e:
            print(f"✗ Error: {e}")
            import traceback
            traceback.print_exc()
            return {}

print("✓ EvaluationRunner ready")

✓ EvaluationRunner ready


# Example Usage

In [6]:
async def run_quick_evaluation():
    """
    Quick evaluation example with proper prompt context.
    Tests the system with refine_prompt and semantic_search tools.
    """
    
    runner = EvaluationRunner()
    
    # User context (like a student)
    user_task = "Learn about machine learning fundamentals"
    user_persona = "Beginner programmer with basic Python knowledge"
    user_mission_objective = "Master neural networks and deep learning concepts"
    
    # A somewhat vague question that would benefit from refinement
    test_question = "gimana machine learning itu?"
    
    reference_answer = "Machine learning is a field of artificial intelligence that enables systems to learn from data without being explicitly programmed."
    
    config = EvaluationConfig(
        model_name=available_models[0]['name'] if available_models else "gpt-4.1-nano",
        use_tools=True,
        max_tool_iterations=3,
    )
    
    result = await runner.evaluate_response(
        config=config,
        test_question=test_question,
        reference_answer=reference_answer,
        user_task=user_task,
        user_persona=user_persona,
        user_mission_objective=user_mission_objective,
        print_steps=True  # Full transparency - show all prompts and tool execution
    )
    
    return result

print("\n" + "="*70)
print("💡 To run evaluation with full prompt transparency:")
print("="*70)
print("result = await run_quick_evaluation()")
print("="*70)


💡 To run evaluation with full prompt transparency:
result = await run_quick_evaluation()


In [9]:
result = await run_quick_evaluation()


🧪 Evaluation Run
Model: gpt-4.1-nano
With Tools: True
Question: gimana machine learning itu?...
Task: Learn about machine learning fundamentals...
Persona: Beginner programmer with basic Python knowledge...
Mission: Master neural networks and deep learning concepts...
----------------------------------------------------------------------
📤 Sending to gpt-4.1-nano...

📋 Evaluation Mode: WITH TOOLS (agentic loop)


📋 SYSTEM PROMPT CONSTRUCTION
# General Prompt
You are a helpful learning assistant with access to two powerful tools:\r                                  +

## Student Learning Profile
# Task
Learn about machine learning fundamentals

# Persona
Beginner programmer with basic Python knowledge

# Mission Objective
Master neural networks and deep learning concepts

  [Iteration 1/3] 🔧 refine_prompt 
      ✅ [STEP 1] Got refine instruction from ChatConfig
      ----------------------------------------------------------------------
      Instruction: Tingkatkan prompt student agar 

In [8]:
result

{'timestamp': '2025-12-30T15:01:20.206466',
 'config': {'model_name': 'gpt-4.1-nano',
  'use_tools': True,
  'with_rag': True,
  'max_tool_iterations': 3,
  'temperature': 0.7,
  'prompt_general_override': None,
  'task_override': None,
  'persona_override': None,
  'mission_objective_override': None,
  'prompt_id_override': None},
 'question': 'gimana machine learning itu?',
 'response': 'Machine learning adalah cabang dari kecerdasan buatan yang memungkinkan komputer belajar dari data dan pengalaman tanpa diprogram secara eksplisit. Dengan menggunakan algoritma dan model statistik, machine learning dapat mengenali pola, membuat prediksi, dan mengambil keputusan berdasarkan data yang diberikan.  \n\nContoh aplikasi machine learning dalam kehidupan sehari-hari meliputi:\n- Sistem rekomendasi seperti yang digunakan oleh Netflix dan Amazon untuk menyarankan film atau produk.\n- Asisten virtual seperti Siri, Google Assistant, dan Alexa yang memahami perintah suara dan merespons.\n- Pengkl